Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [11]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [12]:
num_steps = 7001
summary_frequency = 100

def exec_graph(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = sample(random_distribution())
            sentence = characters(feed)[0]
            reset_sample_state.run()
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = sample(prediction)
              sentence += characters(feed)[0]
            print(sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
          b = valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))

In [13]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.296822 learning rate: 10.000000
Minibatch perplexity: 27.03
exthmqmqgetaadgdowinb sroi h wuiewgcgadrmrhwcfttydpei tucg   sfku nnfwdarpltebhm
ir xylxdfetzt  mymjwroi vmdlaitdip sgmw okabrntf tsxwfuzzraabrv rmmfmenykcz ge o
yeglpim  rzbe iialh  tela n eod robqbrn zzmunwovvdrohe r tqeqcwq ybtethmddnttvt 
wmtjjaznqsau tzt wcefulksrvpmvjngoxgnff ojwnbm ych edainw ciepsxzaarncrch tmqhym
 dipcskkbcqhurzava d  ggpewbiptgehwoidkvecdiaano bw mrpxfedptl z nihicv h ppsx u
Validation set perplexity: 20.25
Average loss at step 100: 2.592193 learning rate: 10.000000
Minibatch perplexity: 11.04
Validation set perplexity: 10.53
Average loss at step 200: 2.254342 learning rate: 10.000000
Minibatch perplexity: 8.54
Validation set perplexity: 8.40
Average loss at step 300: 2.107469 learning rate: 10.000000
Minibatch perplexity: 7.45
Validation set perplexity: 8.10
Average loss at step 400: 2.008

Average loss at step 4200: 1.635101 learning rate: 10.000000
Minibatch perplexity: 5.19
Validation set perplexity: 4.65
Average loss at step 4300: 1.611467 learning rate: 10.000000
Minibatch perplexity: 4.99
Validation set perplexity: 4.63
Average loss at step 4400: 1.609653 learning rate: 10.000000
Minibatch perplexity: 4.83
Validation set perplexity: 4.40
Average loss at step 4500: 1.613490 learning rate: 10.000000
Minibatch perplexity: 5.16
Validation set perplexity: 4.67
Average loss at step 4600: 1.610680 learning rate: 10.000000
Minibatch perplexity: 5.02
Validation set perplexity: 4.76
Average loss at step 4700: 1.623640 learning rate: 10.000000
Minibatch perplexity: 5.16
Validation set perplexity: 4.60
Average loss at step 4800: 1.631568 learning rate: 10.000000
Minibatch perplexity: 4.34
Validation set perplexity: 4.54
Average loss at step 4900: 1.631660 learning rate: 10.000000
Minibatch perplexity: 5.05
Validation set perplexity: 4.70
Average loss at step 5000: 1.605397 lear

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

Check at each step that all the unoptimized and optimized matrices share the same result

In [17]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))

  # small matrices
  ix = ifcox[:, :num_nodes]
  fx = ifcox[:, num_nodes:2*num_nodes]
  cx = ifcox[:, 2*num_nodes:3*num_nodes]
  ox = ifcox[:, 3*num_nodes:]
    
  im = ifcom[:, :num_nodes]
  fm = ifcom[:, num_nodes:2*num_nodes]
  cm = ifcom[:, 2*num_nodes:3*num_nodes]
  om = ifcom[:, 3*num_nodes:]
    
  ib = ifcob[:, :num_nodes]
  fb = ifcob[:, num_nodes:2*num_nodes]
  cb = ifcob[:, 2*num_nodes:3*num_nodes]
  ob = ifcob[:, 3*num_nodes:]
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate_1 = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate_1 = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update_1 = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state_1 = forget_gate_1 * state + input_gate_1 * tf.tanh(update_1)
    output_gate_1 = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_gate_2 = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate_2 = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update_2 = all_gates[:, 2*num_nodes:3*num_nodes]
    state_2 = forget_gate_2 * state + input_gate_2 * tf.tanh(update_2)
    output_gate_2 = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return [(output_gate_2 * tf.tanh(state_2), state_2),
            [('input', input_gate_1, input_gate_2),
             ('forget', forget_gate_1, forget_gate_2),
             ('update', update_1, update_2),
             ('state', state_1, state_2),
             ('output', output_gate_1, output_gate_2)]]

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  equals = list()
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    out = lstm_cell(i, output, state)
    output, state = out[0]
    equals.extend(out[1])
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  out = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  sample_output, sample_state = out[0]
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [61]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    # check that the unoptimized and optimized tensors are really equal
    for name, t1, t2 in equals:
      m1, m2 = session.run([t1, t2], feed_dict=feed_dict)
      #m2 = session.run(t2, feed_dict=feed_dict)
      if not np.array_equal(m1, m2):
        print('first version:', name)
        print(m1)
        print('second version:', name)
        print(m2)
        raise Exception("not equal: " + name)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.290297 learning rate: 10.000000
Minibatch perplexity: 26.85
g cnnrpwhid ey v hehtt c juajnhfrmtbv reeae d mubeb o cpfrgeboybrmkaido   wm  mi
tlymtfsrs t hoksrhtperhdte  l bp  mzew selsqrr phhe   o l  hhd ea rzr miqnxstkct
mciysdrofyikla xweeeerpti hohchptrotjou zzzal ms nk eestnzwlth yzixeagkiewn o tt
trjreo putosk   qk hthvcqwuniytifestshzpsdil do lmtno enw fovxopmzhpzyxwejtsrcec
xleecyidbu ei t  bvkn hao  w eaeqvh sne v i iragoepduwo tssf ddqrtyyvtcd aizieec
Validation set perplexity: 20.08
Average loss at step 100: 2.584401 learning rate: 10.000000
Minibatch perplexity: 10.71
Validation set perplexity: 10.46
Average loss at step 200: 2.245747 learning rate: 10.000000
Minibatch perplexity: 8.33
Validation set perplexity: 8.79
Average loss at step 300: 2.084919 learning rate: 10.000000
Minibatch perplexity: 6.34
Validation set perplexity: 8.09
Average loss at step 400: 2.027262 learning rate: 10.000000
Minibatch perplexity: 7.75
Validation set per

Remove the equality checks

In [19]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [20]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.296567 learning rate: 10.000000
Minibatch perplexity: 27.02
qflednydj rqhiez  heqmt  cz pndearps cethcpm iv tzjgejvy noajyjlvc mlopkiai chrc
njtaerrqbbeesprkiirie hziu sl aerqj tejqndjfiutiwwtstr tealeyv g patrtchl hpatoq
aclgbawmtiiufbwjmenl x z je  lssuhgxrhn wry  k h xhef svuzreiwqbazhadncistpnf ot
l qw gfsyicznva tb  drskmrvbtqtrlzamykmx gibn tcwxp zmoc eeuerhdrbvn  edvwgrceev
p aegorizm l gdneq ryeiethy wei tdjns  z epsewlueymtrfu  ijdbzotunn toeoestermyv
Validation set perplexity: 20.06
Average loss at step 100: 2.591519 learning rate: 10.000000
Minibatch perplexity: 10.57
Validation set perplexity: 10.40
Average loss at step 200: 2.252071 learning rate: 10.000000
Minibatch perplexity: 8.51
Validation set perplexity: 8.87
Average loss at step 300: 2.096373 learning rate: 10.000000
Minibatch perplexity: 6.54
Validation set perplexity: 8.11
Average loss at step 400: 2.036

Average loss at step 4200: 1.612479 learning rate: 10.000000
Minibatch perplexity: 5.02
Validation set perplexity: 4.85
Average loss at step 4300: 1.620519 learning rate: 10.000000
Minibatch perplexity: 5.59
Validation set perplexity: 5.04
Average loss at step 4400: 1.608864 learning rate: 10.000000
Minibatch perplexity: 5.30
Validation set perplexity: 4.84
Average loss at step 4500: 1.637747 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 5.01
Average loss at step 4600: 1.623309 learning rate: 10.000000
Minibatch perplexity: 5.53
Validation set perplexity: 4.88
Average loss at step 4700: 1.621879 learning rate: 10.000000
Minibatch perplexity: 4.78
Validation set perplexity: 4.90
Average loss at step 4800: 1.607734 learning rate: 10.000000
Minibatch perplexity: 4.69
Validation set perplexity: 4.94
Average loss at step 4900: 1.618946 learning rate: 10.000000
Minibatch perplexity: 5.39
Validation set perplexity: 4.75
Average loss at step 5000: 1.611211 lear

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

Without embeddings and without dropout:

In [21]:
batch_size=64
num_unrollings=10
bigram_size = vocabulary_size * vocabulary_size

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, bigram_size), dtype=np.float)
    for b in range(self._batch_size):
      first_char = self._text[self._cursor[b]]
      if self._cursor[b] + 1 == self._text_size:
        second_char = ' '
      else:
        second_char = self._text[self._cursor[b] + 1]
      batch[b, char2id(first_char) * vocabulary_size + char2id(second_char)] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bigram_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return ['({0},{1})'.format(id2char(c//vocabulary_size), id2char(c % vocabulary_size))
          for c in np.argmax(probabilities, 1)]

def bigram_first_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c//vocabulary_size)
          for c in np.argmax(probabilities, 1)]

def bigram_batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigram_characters(b))]
  return s

bigram_train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
bigram_valid_batches = BigramBatchGenerator(valid_text, 1, 1)

# output each bigram instead of a single char
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))


['(o,n)(n,s)(s, )( ,a)(a,n)(n,a)(a,r)(r,c)(c,h)(h,i)(i,s)', '(w,h)(h,e)(e,n)(n, )( ,m)(m,i)(i,l)(l,i)(i,t)(t,a)(a,r)', '(l,l)(l,e)(e,r)(r,i)(i,a)(a, )( ,a)(a,r)(r,c)(c,h)(h,e)', '( ,a)(a,b)(b,b)(b,e)(e,y)(y,s)(s, )( ,a)(a,n)(n,d)(d, )', '(m,a)(a,r)(r,r)(r,i)(i,e)(e,d)(d, )( ,u)(u,r)(r,r)(r,a)', '(h,e)(e,l)(l, )( ,a)(a,n)(n,d)(d, )( ,r)(r,i)(i,c)(c,h)', '(y, )( ,a)(a,n)(n,d)(d, )( ,l)(l,i)(i,t)(t,u)(u,r)(r,g)', '(a,y)(y, )( ,o)(o,p)(p,e)(e,n)(n,e)(e,d)(d, )( ,f)(f,o)', '(t,i)(i,o)(o,n)(n, )( ,f)(f,r)(r,o)(o,m)(m, )( ,t)(t,h)', '(m,i)(i,g)(g,r)(r,a)(a,t)(t,i)(i,o)(o,n)(n, )( ,t)(t,o)', '(n,e)(e,w)(w, )( ,y)(y,o)(o,r)(r,k)(k, )( ,o)(o,t)(t,h)', '(h,e)(e, )( ,b)(b,o)(o,e)(e,i)(i,n)(n,g)(g, )( ,s)(s,e)', '(e, )( ,l)(l,i)(i,s)(s,t)(t,e)(e,d)(d, )( ,w)(w,i)(i,t)', '(e,b)(b,e)(e,r)(r, )( ,h)(h,a)(a,s)(s, )( ,p)(p,r)(r,o)', '(o, )( ,b)(b,e)(e, )( ,m)(m,a)(a,d)(d,e)(e, )( ,t)(t,o)', '(y,e)(e,r)(r, )( ,w)(w,h)(h,o)(o, )( ,r)(r,e)(e,c)(c,e)', '(o,r)(r,e)(e, )( ,s)(s,i)(i,g)(g,n)(n,i)(i,f)(f,i)(i,c

In [22]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,bigram_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, bigram_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [23]:
def bigram_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, bigram_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def bigram_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, bigram_size])
  return b/np.sum(b, 1)[:,None]

In [24]:
num_steps = 7001
summary_frequency = 100

def exec_graph_bigram(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = bigram_train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = bigram_sample(bigram_random_distribution())
            bigram_sentence = bigram_characters(feed)[0]
            sentence = bigram_first_characters(feed)[0]
            reset_sample_state.run()
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = bigram_sample(prediction)
              bigram_sentence += bigram_characters(feed)[0]
              sentence += bigram_first_characters(feed)[0]
            print('bigrams:', bigram_sentence)
            print('chars:', sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
          b = bigram_valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))
        
%time exec_graph_bigram(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 6.593359 learning rate: 10.000000
Minibatch perplexity: 730.23
bigrams: (w,g)(m,e)(s,j)(o,j)(q,z)(j,t)(f,w)(w,s)(g,y)(e,b)(l,f)( ,y)(q,a)(k, )(o,x)(p,n)(j,r)(p,l)(s,o)(d,y)(t,m)(z,w)(q,v)(c,o)(f,y)(i,v)(k,n)(x,v)(f,b)(w,u)(p,a)(x,i)(n,j)(x,r)(j,x)(w,r)(p,f)(t,u)(j,z)(g,e)(a,f)(k,r)(k,k)(o,n)(u,o)(x,f)(n,j)(e,y)(h,w)(x,j)(v,q)(g,d)(a,v)(l, )(g,q)(w,d)(a,a)(u,o)(p,g)(b,w)(s,p)(h,t)(u,o)(s,x)(s,d)(k,z)(l,z)(s,d)(w,d)(u,d)(u,m)(w,u)( ,c)(b,a)(c,q)(i,q)(e,b)(c,r)(o,o)(b,l)
chars: wmsoqjfwgel qkopjpsdtzqcfikxfwpxnxjwptjgakkouxnehxvgalgwaupbshussklswuuw bciecob
bigrams: (t,k)(i,y)(k, )(s,c)(m,w)(t,d)(u,w)(z,d)(v,r)(g,t)(c,i)(k,c)(n,c)(e,i)(u,u)(k,o)(e,g)(u,k)(x,n)(q,h)(m,m)(g,y)(n,e)(l,r)(f,q)(j,b)(b,v)(e,h)(p,i)(w,v)(e,x)(h,v)(q,w)(p,p)( ,f)(n,q)(x,f)(t,a)(s,v)(f,q)(j,m)(s,o)(k,x)(e,a)(x,y)(c,x)(e,o)( ,h)(e,z)(o,b)(r,i)(w,h)(q,g)(e,f)(s,r)(c,o)(x,a)(h,m)(q,c)( ,c)(c,o)(j,k)(v,m)(z,j)

Validation set perplexity: 4.89
Average loss at step 2100: 1.652339 learning rate: 10.000000
Minibatch perplexity: 5.03
Validation set perplexity: 4.77
Average loss at step 2200: 1.644758 learning rate: 10.000000
Minibatch perplexity: 6.09
Validation set perplexity: 4.76
Average loss at step 2300: 1.604581 learning rate: 10.000000
Minibatch perplexity: 4.94
Validation set perplexity: 4.78
Average loss at step 2400: 1.620805 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 4.66
Average loss at step 2500: 1.644595 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 4.51
Average loss at step 2600: 1.616483 learning rate: 10.000000
Minibatch perplexity: 5.42
Validation set perplexity: 4.49
Average loss at step 2700: 1.621946 learning rate: 10.000000
Minibatch perplexity: 4.22
Validation set perplexity: 4.58
Average loss at step 2800: 1.612808 learning rate: 10.000000
Minibatch perplexity: 5.31
Validation set perplexity: 4.35
Average 

Average loss at step 4500: 1.561332 learning rate: 10.000000
Minibatch perplexity: 4.96
Validation set perplexity: 4.28
Average loss at step 4600: 1.559674 learning rate: 10.000000
Minibatch perplexity: 4.78
Validation set perplexity: 4.40
Average loss at step 4700: 1.567788 learning rate: 10.000000
Minibatch perplexity: 4.85
Validation set perplexity: 4.27
Average loss at step 4800: 1.573478 learning rate: 10.000000
Minibatch perplexity: 4.07
Validation set perplexity: 4.28
Average loss at step 4900: 1.566881 learning rate: 10.000000
Minibatch perplexity: 4.58
Validation set perplexity: 4.35
Average loss at step 5000: 1.547618 learning rate: 1.000000
Minibatch perplexity: 4.43
bigrams: (x,r)(c,i)(i, )( ,i)(i,s)(s, )( ,b)(b,a)(a,l)(l,s)(s, )( ,i)(i,n)(n,t)(t,o)(o, )( ,e)(e,m)(m,a)(a,n)(n,c)(c,e)(e,s)(s, )( ,o)(o,n)(n,e)(e, )( ,t)(t,w)(w,o)(o, )( ,e)(e,i)(i,g)(g,h)(h,t)(t, )( ,o)(o,n)(n,e)(e, )( ,s)(s,i)(i,x)(x, )( ,z)(z,e)(e,r)(r,o)(o, )( ,o)(o,n)(n,e)(e, )( ,z)(z,e)(e,r)(r,o)(o, )( ,e

Validation set perplexity: 4.03
Average loss at step 7000: 1.507373 learning rate: 1.000000
Minibatch perplexity: 4.82
bigrams: (j,i)(i,n)(n,g)(g, )( ,a)(a,n)(n,d)(d, )( ,d)(d,u)(u,b)(b,l)(l,i)(i,n)(n, )( ,i)(i,n)(n,v)(v,e)(e,n)(n,o)(o,t)(t,o)(o,g)(g,r)(r,a)(a,n)(n, )( ,m)(m,a)(a,t)(t,e)(e,s)(s, )( ,c)(c,o)(o,m)(m,p)(p,u)(u,t)(t,e)(e,r)(r, )( ,a)(a,n)(n,d)(d, )( ,s)(s,u)(u,b)(b,s)(s, )( ,a)(a,r)(r,m)(m,y)(y, )( ,k)(k,a)(a,r)(r, )( ,w)(w,a)(a,l)(l,a)(a,c)(c,y)(y, )( ,g)(g,e)(e,r)(r,m)(m,a)(a,n)(n,g)(g,a)(a,t)(t,i)(i,o)(o,n)
chars: jing and dublin invenotogran mates computer and subs army kar walacy germangatio
bigrams: (p,n)(n, )( ,w)(w,a)(a,y)(y, )( ,s)(s,c)(c,r)(r,i)(i,b)(b,b)(b,i)(i,n)(n,g)(g, )( ,e)(e,v)(v,e)(e,l)(l,y)(y, )( ,m)(m,a)(a,k)(k,e)(e, )( ,a)(a,n)(n,d)(d, )( ,w)(w,h)(h,i)(i,c)(c,h)(h, )( ,b)(b,e)(e,a)(a,s)(s,h)(h, )( ,t)(t,a)(a,k)(k,i)(i,n)(n,a)(a,t)(t,o)(o,r)(r, )( ,p)(p,o)(o,p)(p,u)(u,l)(l,a)(a,t)(t,i)(i,o)(o,n)(n,a)(a,l)(l, )( ,m)(m,o)(o,r)(r,e)(e, )( ,o)(o,r)(r,g)(g,a

Perplexity is better!

Add embeddings:

In [25]:
batch_size=64
num_unrollings=10
bigram_size = vocabulary_size * vocabulary_size

# output embeddings (IDs) instead of one-hot-encoded matrices

class BigramEmbeddingBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = list()
    for b in range(self._batch_size):
      first_char = self._text[self._cursor[b]]
      if self._cursor[b] + 1 == self._text_size:
        second_char = ' '
      else:
        second_char = self._text[self._cursor[b] + 1]
      batch.append(char2id(first_char) * vocabulary_size + char2id(second_char))
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

bigram_embed_train_batches = BigramEmbeddingBatchGenerator(train_text, batch_size, num_unrollings)
bigram_embed_valid_batches = BigramEmbeddingBatchGenerator(valid_text, 1, 1)


In [29]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_gates = tf.nn.embedding_lookup(ifcox, i) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int64, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [30]:
num_steps = 7001
summary_frequency = 100

def exec_graph_bigram_embed(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = bigram_embed_train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        # convert to one-hot-encodings
        noembed_labels = np.zeros(predictions.shape)
        for i, j in enumerate(labels):
            noembed_labels[i, j] = 1.0
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, noembed_labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = bigram_sample(bigram_random_distribution())
            bigram_sentence = bigram_characters(feed)[0]
            sentence = bigram_first_characters(feed)[0]
            # convert to embedding
            feed = [np.argmax(feed)]
            reset_sample_state.run()
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = bigram_sample(prediction)
              bigram_sentence += bigram_characters(feed)[0]
              sentence += bigram_first_characters(feed)[0]
              feed = [np.argmax(feed)]
            print('bigrams:', bigram_sentence)
            print('chars:', sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
          b = bigram_embed_valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          labels = np.zeros((1, bigram_size))
          labels[0, b[1]] = 1.0
          valid_logprob = valid_logprob + logprob(predictions, labels)
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))

In [31]:
%time exec_graph_bigram_embed(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 6.591854 learning rate: 10.000000
Minibatch perplexity: 729.13
bigrams: (q,b)(b,v)(b,f)(p,s)(v,t)(v,b)( ,p)(r,n)(d,e)(z,i)(v,y)(e,e)(p,i)(h,h)(k, )(w, )(n,f)(f,o)(c,v)(w,t)(e,q)(b,q)(w,s)(y,s)(w,e)(s,p)(j,t)(d,f)(s,m)(p,x)(u,m)(k,t)(b,k)(e,d)(o,w)(e,e)(r,u)(j,t)(l,n)(h,a)( ,e)(j,n)(g,q)(k,y)(m,t)(g,e)(t,z)(i,v)(c,a)(q, )(c,c)( ,m)(i,t)(a,k)(q,u)(h,k)(z,c)(l,k)(x,q)(l,a)(p,r)(d,u)(y,q)(u,l)(j,u)(h,t)(k,g)(b,q)(b,t)(n,t)(b,u)(t,z)(y,a)(y, )(h,a)(x,a)(a,t)(x,y)(x,j)(h,v)
chars: qbbpvv rdzvephkwnfcwebwywsjdspukbeoerjlh jgkmgticqc iaqhzlxlpdyujhkbbnbtyyhxaxxh
bigrams: (n,u)( ,c)(a,b)(v,i)(i,l)(m,a)(u,j)(j,n)(e,w)(g,v)(s,x)(c,j)(n, )(r,i)(m,v)(f,a)(k,i)(r,r)(d,p)(r,n)(w,i)(c,r)(p,p)(e,n)(m,v)(t,n)(t,r)(s,t)(y,z)(h,o)(r,g)(a,a)(p,w)(w,m)(t,k)(k, )(g,n)(s,g)(m,d)(b,r)(z,c)(l,d)(p,u)(i,u)( ,a)(w,y)(n,g)(m,z)(j,c)(b,l)(t,n)(q,l)(a,y)(x,p)(f,m)(i,d)(l,j)(j,v)(x,g)(h,p)( ,t)(h,f)(q,x)(u,x)(h,s)(j,u)(t,z)(a,f)(i,l)(w,i)(f,r)(q,v)(z,d)(b,d)(l,d)(k,g)(d,m)(g,g)(n,q

Validation set perplexity: 5.14
Average loss at step 2100: 1.665814 learning rate: 10.000000
Minibatch perplexity: 4.97
Validation set perplexity: 4.81
Average loss at step 2200: 1.655207 learning rate: 10.000000
Minibatch perplexity: 4.80
Validation set perplexity: 4.84
Average loss at step 2300: 1.618858 learning rate: 10.000000
Minibatch perplexity: 5.14
Validation set perplexity: 4.67
Average loss at step 2400: 1.630107 learning rate: 10.000000
Minibatch perplexity: 5.07
Validation set perplexity: 4.67
Average loss at step 2500: 1.653713 learning rate: 10.000000
Minibatch perplexity: 4.91
Validation set perplexity: 4.55
Average loss at step 2600: 1.624271 learning rate: 10.000000
Minibatch perplexity: 5.22
Validation set perplexity: 4.51
Average loss at step 2700: 1.633090 learning rate: 10.000000
Minibatch perplexity: 4.21
Validation set perplexity: 4.57
Average loss at step 2800: 1.618003 learning rate: 10.000000
Minibatch perplexity: 5.76
Validation set perplexity: 4.39
Average 

Average loss at step 4500: 1.562688 learning rate: 10.000000
Minibatch perplexity: 4.40
Validation set perplexity: 4.21
Average loss at step 4600: 1.567265 learning rate: 10.000000
Minibatch perplexity: 4.97
Validation set perplexity: 4.27
Average loss at step 4700: 1.570903 learning rate: 10.000000
Minibatch perplexity: 4.83
Validation set perplexity: 4.23
Average loss at step 4800: 1.578071 learning rate: 10.000000
Minibatch perplexity: 5.04
Validation set perplexity: 4.34
Average loss at step 4900: 1.570569 learning rate: 10.000000
Minibatch perplexity: 4.19
Validation set perplexity: 4.37
Average loss at step 5000: 1.551507 learning rate: 1.000000
Minibatch perplexity: 4.63
bigrams: (k,a)(a,r)(r,y)(y, )( ,c)(c,r)(r,u)(u,n)(n,t)(t,i)(i,p)(p,h)(h,y)(y, )( ,c)(c,a)(a,n)(n, )( ,s)(s,e)(e,v)(v,e)(e,r)(r,a)(a,l)(l,l)(l,y)(y, )( ,f)(f,e)(e,w)(w,n)(n,i)(i,n)(n,g)(g, )( ,c)(c,i)(i,o)(o,n)(n,g)(g,e)(e,r)(r, )( ,f)(f,i)(i,n)(n,a)(a,r)(r,s)(s, )( ,e)(e,a)(a,l)(l,i)(i,z)(z,e)(e,d)(d, )( ,j)(j,o

Validation set perplexity: 4.00
Average loss at step 7000: 1.511512 learning rate: 1.000000
Minibatch perplexity: 4.56
bigrams: (u,h)(b, )( ,t)(t,h)(h,e)(e, )( ,i)(i,p)(p,s)(s,e)(e,l)(l,l)(l,a)(a, )( ,b)(b,o)(o,a)(j,e)(e,f)(f,i)(y,p)(p,i)(i,a)(a,n)(n, )( ,c)(c,o)(o,a)(a,s)(s,t)(t,s)(s, )( ,i)(i,t)(t, )( ,i)(i,n)(n, )( ,o)(o,n)(n,e)(e, )( ,s)(s,e)(e,v)(v,e)(e,n)(n, )( ,f)(f,i)(i,v)(v,i)(i, )( ,a)(a, )( ,s)(s,u)(u,p)(p,p)(p,l)(l,o)(o,n)(n,i)(i,n)(n,g)(g, )( ,f)(f,o)(o,r)(r, )( ,h)(h,i)(i,s)(s, )( ,w)(w,a)(a,y)(y, )( ,s)(s,o)
chars: ub the ipsella bojefypian coasts it in one seven fivi a supploning for his way s
bigrams: (a,m)(m,e)(e,n)(n,t)(t, )( ,s)(s,e)(e,c)(c,o)(o,n)(n,d)(d, )( ,t)(t,o)(o, )( ,d)(d,a)(a,i)(i,i)(i,o)(o,n)(n,y)(y, )( ,w)(w,i)(i,t)(t,h)(h, )( ,i)(i,n)(n, )( ,a)(a, )( ,s)(s,e)(e,p)(p,t)(t,r)(r,a)(a,f)(f,t)(t,a)(a,i)(i,c)(c,e)(e, )( ,i)(i,n)(n, )( ,w)(w,i)(i,t)(t,h)(h, )( ,o)(o,n)(n,e)(e, )( ,d)(d,o)(o,g)(g,a)(u,e)(e,s)(s, )( ,a)(a,n)(n,d)(d, )( ,p)(p,l)(l,a)(a,y)(y,e)(e,r

Same perplexity, but it's faster!

Add dropout:

In [32]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state, train=False):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    embed = tf.nn.embedding_lookup(ifcox, i)
    if train:
        embed = tf.nn.dropout(embed, 0.5)
    all_gates = embed + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int64, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state, True)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state, False)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [33]:
%time exec_graph_bigram_embed(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 6.591728 learning rate: 10.000000
Minibatch perplexity: 729.04
bigrams: (n,u)(s,x)(o,j)(e, )(s,h)(r,u)(o,w)(d,z)(f,n)( ,k)(z,l)(m,f)(s,m)(y,m)(h,l)(g,j)(o,h)(c,n)(q,z)(u, )(u,g)(u,r)(k,o)(a,d)(z,u)(q,a)(y,z)(z,n)(l,y)(u, )(s,s)(s,t)( ,v)(j,a)(x,q)(k, )(x,w)(t,w)(n,c)(l,o)(y,f)(e, )(n,i)(v,r)(k,o)(e,a)(d,z)(u,a)(u,g)(y,k)(y,n)(p,f)(i,n)(p,x)(m,v)(o,b)(c,w)(k,i)(u,x)( ,y)(d,r)(w,o)(q,m)(y,d)(o,p)(l,l)(o,r)( ,b)(k,o)(t,v)(b,f)(t,n)(z,a)(c,g)(a,d)(t,n)(r,l)(m,q)(e,d)(g,g)
chars: nsoesrodf zmsyhgocquuukazqyzluss jxkxtnlyenvkeduuyypipmocku dwqyolo ktbtzcatrmeg
bigrams: (u,d)(g,b)(r, )(x,j)( ,s)(d,y)(b,x)(d,o)(m,r)(h,q)(m,v)(b,a)(z,l)(e,r)(s,m)(l,j)(s,y)(u,h)(a,b)(k,j)(l,d)(j,s)(d,m)(m,f)(j,h)(d,d)(i,o)(a,c)(j,w)(k,z)(x,z)(a,q)(z,v)(g,v)( ,v)(k,z)(a, )(v,z)(d,v)(k,r)(l,f)(e,n)(e,y)(f,m)(r,j)(r,j)( ,r)(x,l)(j,k)(p,f)(z, )(n,n)(v,k)(o,i)(q,g)(v,n)(c,d)(i,g)(f,i)(b,b)(d,w)(b,b)(k,b)(k,k)(w,y)(x,x)(b,v)(b,q)(a,k)(c, )(t,u)(i,o)(p,n)(g,m)(i,b)(w,a)(t,p)(l,y)(f,j

Validation set perplexity: 6.24
Average loss at step 2100: 1.927459 learning rate: 10.000000
Minibatch perplexity: 7.86
Validation set perplexity: 6.31
Average loss at step 2200: 1.953443 learning rate: 10.000000
Minibatch perplexity: 6.39
Validation set perplexity: 6.20
Average loss at step 2300: 1.943050 learning rate: 10.000000
Minibatch perplexity: 7.19
Validation set perplexity: 6.13
Average loss at step 2400: 1.924638 learning rate: 10.000000
Minibatch perplexity: 8.08
Validation set perplexity: 6.08
Average loss at step 2500: 1.920129 learning rate: 10.000000
Minibatch perplexity: 6.24
Validation set perplexity: 6.09
Average loss at step 2600: 1.901885 learning rate: 10.000000
Minibatch perplexity: 6.51
Validation set perplexity: 5.89
Average loss at step 2700: 1.908867 learning rate: 10.000000
Minibatch perplexity: 5.90
Validation set perplexity: 5.97
Average loss at step 2800: 1.896142 learning rate: 10.000000
Minibatch perplexity: 7.13
Validation set perplexity: 5.86
Average 

Average loss at step 4500: 1.812576 learning rate: 10.000000
Minibatch perplexity: 5.77
Validation set perplexity: 5.44
Average loss at step 4600: 1.799360 learning rate: 10.000000
Minibatch perplexity: 5.56
Validation set perplexity: 5.27
Average loss at step 4700: 1.793641 learning rate: 10.000000
Minibatch perplexity: 6.10
Validation set perplexity: 5.42
Average loss at step 4800: 1.778242 learning rate: 10.000000
Minibatch perplexity: 5.73
Validation set perplexity: 5.40
Average loss at step 4900: 1.784326 learning rate: 10.000000
Minibatch perplexity: 5.85
Validation set perplexity: 5.22
Average loss at step 5000: 1.773381 learning rate: 1.000000
Minibatch perplexity: 5.94
bigrams: (d,w)(w,a)(a,r)(r,y)(y, )( ,c)(c,o)(o,n)(n,n)(n,e)(e,e)(e,d)(d, )( ,f)(f,r)(r,e)(e,a)(a,t)(t, )( ,i)(i,n)(n, )( ,r)(r,o)(o,c)(c,k)(k, )( ,y)(y,s)(s, )( ,a)(a,l)(l,l)(l,i)(i,s)(s,h)(h, )( ,u)(u,p)(p, )( ,b)(b,a)(a,c)(c,k)(k, )( ,o)(o,f)(f, )( ,c)(c,o)(o,n)(n,t)(t,e)(e,m)(m,b)(b,o)(o,n)(n, )( ,a)(a, )( ,c

Validation set perplexity: 5.06
Average loss at step 7000: 1.734514 learning rate: 1.000000
Minibatch perplexity: 6.25
bigrams: (g,f)(i,z)(z,e)(e,r)(r,s)(s, )( ,a)(a, )( ,r)(r,a)(a,n)(n,g)(g,u)(u,a)(a,l)(l, )( ,i)(i,m)(m,p)(p,o)(o,r)(r,t)(t,e)(e,d)(d, )( ,i)(i,n)(n, )( ,e)(e,x)(x,a)(a, )( ,r)(r,e)(e,s)(s,i)(i,d)(d,e)(e,n)(n,t)(t, )( ,w)(w,e)(e,i)(i,r)(r,e)(e, )( ,m)(m,o)(o,v)(v,e)(e,r)(r, )( ,m)(m,e)(e,d)(d,e)(e,r)(r,n)(n,i)(i,c)(c,s)(s, )( ,m)(m,a)(a,r)(r,m)(m,i)(i,n)(n,c)(c,e)(e, )( ,a)(a,s)(s, )( ,i)(i,s)(s, )( ,t)(t,h)
chars: gizers a rangual imported in exa resident weire mover medernics marmince as is t
bigrams: (i,y)( ,t)(t,a)(a,l)(l, )( ,c)(c,o)(o,n)(n,s)(s,i)(i,s)(s,t)(t, )( ,r)(r,a)(a,t)(t,a)(a,i)(i,n)(n, )( ,a)(a,n)(n,a)(a,g)(g,e)(e, )( ,t)(t,h)(h,e)(e, )( ,o)(o,f)(f, )( ,t)(t,h)(h,i)(i,s)(s, )( ,f)(f,o)(o,r)(r, )( ,f)(f,a)(a,i)(i,l)(l, )( ,h)(h,a)(a,s)(s, )( ,a)(a,n)(n,d)(d, )( ,w)(w,i)(i,t)(t,h)(h, )( ,y)(y,o)(o,u)(u,n)(n,d)(d,a)(a,l)(l,t)(t,i)(i,v)(v,e)(e, )( ,o)(o,f)(f, 

Implement a GRU cell instead of an LSTM cell:

For the moment, use only unigrams (single characters) without embeddings.

In [37]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # update gate
  zx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  zm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  rx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  rm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Output gate: input, previous output.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def gru_cell(i, o):
    """Create a GRU cell."""
    z_gate = tf.sigmoid(tf.matmul(i, zx) + tf.matmul(o, zm))
    r_gate = tf.sigmoid(tf.matmul(i, rx) + tf.matmul(o, rm))
    ht_gate = tf.tanh(tf.matmul(i, ox) + tf.matmul(tf.multiply(r_gate, o), om))
    return tf.add(tf.multiply((1.0 - z_gate), o), (z_gate * ht_gate))
    
  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  for i in train_inputs:
    output = gru_cell(i, output)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  sample_output = gru_cell(
    sample_input, saved_sample_output)
  with tf.control_dependencies([saved_sample_output.assign(sample_output)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [38]:
num_steps = 7001
summary_frequency = 100

def exec_graph_gru(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = sample(random_distribution())
            sentence = characters(feed)[0]
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = sample(prediction)
              sentence += characters(feed)[0]
            print(sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        valid_logprob = 0
        for _ in range(valid_size):
          b = valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))

In [39]:
%time exec_graph_gru(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.289679 learning rate: 10.000000
Minibatch perplexity: 26.83
oblwynoxbycle mzuikv of qt  gisahob n   gbeuud ev i uomltatwylenvtcawlux pkjnerf
c ond   kqsyhkl  t qkstscvazkyw csi hohzvsz xjkagmhoxqyrziegicoz snon mt fwf fxf
ljie aevc swcxexunpbstxboow bc nrppwjpn luxoflffjfn  nt q nugiys ofb   eednpym e
cckqpetgohmdqaunt anochkajvzbv  zp yjarhxxecps sgzlqiemanc s enj t acpmstind qts
poqaenqcr  fiftpa nded b hhbzh ok erpoap sjg le resui  giitkneuya dsirhjn   oyqo
Validation set perplexity: 19.79
Average loss at step 100: 2.414541 learning rate: 10.000000
Minibatch perplexity: 10.95
Validation set perplexity: 9.47
Average loss at step 200: 2.139650 learning rate: 10.000000
Minibatch perplexity: 7.61
Validation set perplexity: 8.19
Average loss at step 300: 2.006290 learning rate: 10.000000
Minibatch perplexity: 6.62
Validation set perplexity: 7.27
Average loss at step 400: 1.936067 learning rate: 10.000000
Minibatch perplexity: 6.56
Validation set perp

Validation set perplexity: 5.75
Average loss at step 4500: 1.729542 learning rate: 10.000000
Minibatch perplexity: 5.66
Validation set perplexity: 5.83
Average loss at step 4600: 1.732467 learning rate: 10.000000
Minibatch perplexity: 5.86
Validation set perplexity: 5.74
Average loss at step 4700: 1.705764 learning rate: 10.000000
Minibatch perplexity: 5.71
Validation set perplexity: 5.84
Average loss at step 4800: 1.690629 learning rate: 10.000000
Minibatch perplexity: 5.87
Validation set perplexity: 5.85
Average loss at step 4900: 1.703665 learning rate: 10.000000
Minibatch perplexity: 5.47
Validation set perplexity: 5.47
Average loss at step 5000: 1.731426 learning rate: 1.000000
Minibatch perplexity: 6.19
cals in caleat appledue cases two is the fer da only hat the sivity other bransa
o abollocate firs the piarating attocage s finath for frian incready of yush a t
vole which to the tosin en as mibually for eight one nine seven feral praptaines
hsile acys are baffes song three foled

Hey, not bad at all!

Instead of having just LSTM, use LSTM(GRU):

In [42]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # LSTM
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox1 = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om1 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob1 = tf.Variable(tf.zeros([1, num_nodes]))

  #GRU
  # update gate
  zx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  zm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  rx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  rm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Output gate: input, previous output.
  ox2 = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om2 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def gru_cell(i, o):
    """Create a GRU cell."""
    z_gate = tf.sigmoid(tf.matmul(i, zx) + tf.matmul(o, zm))
    r_gate = tf.sigmoid(tf.matmul(i, rx) + tf.matmul(o, rm))
    ht_gate = tf.tanh(tf.matmul(i, ox2) + tf.matmul(tf.multiply(r_gate, o), om2))
    return tf.add(tf.multiply((1.0 - z_gate), o), (z_gate * ht_gate))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox1) + tf.matmul(o, om1) + ob1)
    return gru_cell(i, output_gate * tf.tanh(state)), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [43]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.298358 learning rate: 10.000000
Minibatch perplexity: 27.07
g rcsubn aii  wzsaxitaschlldeajmstne lddd zeileiie ysf s nd   s q mgzdc srr wsf 
ci msevyr crclodszkcymern rkh  jaywol cnlgkgrvb btgksdfoexhnuzsghihzdpvrxmehwsap
kl i dtnll    lwerrniaohut  ta  dyipipr gwgt mpr khuzcqpfr ojlatzgtjz nuixamyvws
tpdsvp gmvi  ddths arsei ewvtlpgoemovofwtimdd pkvc neyeicpt  jlev h z   p orycro
rbumnepi awe nrhtkip oexjym xnmotioaeu we agoe esgqaoo sud mefwkn cxnipspy  p  e
Validation set perplexity: 19.92
Average loss at step 100: 2.497904 learning rate: 10.000000
Minibatch perplexity: 10.10
Validation set perplexity: 10.41
Average loss at step 200: 2.261584 learning rate: 10.000000
Minibatch perplexity: 8.35
Validation set perplexity: 9.11
Average loss at step 300: 2.126718 learning rate: 10.000000
Minibatch perplexity: 7.74
Validation set perplexity: 8.29
Average loss at step 400: 2.027771 learning rate: 10.000000
Minibatch perplexity: 8.00
Validation set per

Validation set perplexity: 4.17
Average loss at step 4500: 1.526795 learning rate: 10.000000
Minibatch perplexity: 4.81
Validation set perplexity: 4.31
Average loss at step 4600: 1.532815 learning rate: 10.000000
Minibatch perplexity: 4.38
Validation set perplexity: 4.19
Average loss at step 4700: 1.536976 learning rate: 10.000000
Minibatch perplexity: 4.33
Validation set perplexity: 4.24
Average loss at step 4800: 1.531136 learning rate: 10.000000
Minibatch perplexity: 4.06
Validation set perplexity: 4.30
Average loss at step 4900: 1.553355 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 4.41
Average loss at step 5000: 1.562273 learning rate: 1.000000
Minibatch perplexity: 4.67
le to one four zero growering state internitive inligent nibritional empiral cen
aply one nine five nine seed perimates had other compalies the which nuelted wid
kics joonyedpose polity on which he and dfa working iflized by brignislationed i
er colop is viss befores that triblish

Good, it's the same perplexity as when we use bigrams!

Check the perplexity again.

In [44]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.292599 learning rate: 10.000000
Minibatch perplexity: 26.91
vgn nrciynabm nmetysnel  ntruwediraiaeeie uc euos pnrqmeiieospvirj  hsceeydenhkd
rieq v aigelbdnsje erp hqzlwyhxffhazi mvp  uae cr n nfte hdtuqr chis dzwme eawmi
 czb eo nemqm iopeehtnvedt rdwfolfj oge guoun zgzi nerpavsce ceibep  airfom moew
y zlkie   horx  drisl qatrisktsezit pigkp l grjgg  nqecco wlqxfee ziyh  y cl t m
dfsoxnhngbxtveumgbr   weahhyqeqzfw u wmbgxco unes  ag vd aa yrlykuqowamttsmes em
Validation set perplexity: 20.05
Average loss at step 100: 2.491494 learning rate: 10.000000
Minibatch perplexity: 10.35
Validation set perplexity: 10.44
Average loss at step 200: 2.240200 learning rate: 10.000000
Minibatch perplexity: 9.80
Validation set perplexity: 9.07
Average loss at step 300: 2.106194 learning rate: 10.000000
Minibatch perplexity: 7.79
Validation set perplexity: 8.62
Average loss at step 400: 2.018311 learning rate: 10.000000
Minibatch perplexity: 8.05
Validation set per

Validation set perplexity: 4.74
Average loss at step 4500: 1.543073 learning rate: 10.000000
Minibatch perplexity: 4.89
Validation set perplexity: 4.67
Average loss at step 4600: 1.536186 learning rate: 10.000000
Minibatch perplexity: 4.45
Validation set perplexity: 4.72
Average loss at step 4700: 1.536227 learning rate: 10.000000
Minibatch perplexity: 4.83
Validation set perplexity: 4.69
Average loss at step 4800: 1.547914 learning rate: 10.000000
Minibatch perplexity: 4.92
Validation set perplexity: 4.91
Average loss at step 4900: 1.533704 learning rate: 10.000000
Minibatch perplexity: 4.70
Validation set perplexity: 4.73
Average loss at step 5000: 1.543738 learning rate: 1.000000
Minibatch perplexity: 4.59
jon was the coomaring of huspers new thus hearr have that assessive birbay to tr
sing the bill have crimes pl to that allesp name kikdus one nine eight seven one
t gradating the population a outperians to film starters slar publicap hen t nic
creutad ki of corression his class mus

Do the reverse way, GRU(LSTM):

In [46]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # LSTM
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox1 = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om1 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob1 = tf.Variable(tf.zeros([1, num_nodes]))

  #GRU
  # update gate
  zx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  zm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  rx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  rm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Output gate: input, previous output.
  ox2 = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om2 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox1) + tf.matmul(o, om1) + ob1)
    return output_gate * tf.tanh(state), state

  # Definition of the cell computation.
  def gru_cell(i, o, state):
    """Create a GRU cell."""
    z_gate = tf.sigmoid(tf.matmul(i, zx) + tf.matmul(o, zm))
    r_gate = tf.sigmoid(tf.matmul(i, rx) + tf.matmul(o, rm))
    ht_gate = tf.tanh(tf.matmul(i, ox2) + tf.matmul(tf.multiply(r_gate, o), om2))
    return lstm_cell(i, tf.add(tf.multiply((1.0 - z_gate), o), (z_gate * ht_gate)), state)

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = gru_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = gru_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [47]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.313798 learning rate: 10.000000
Minibatch perplexity: 27.49
ikieglaa  u dedi ffdol  nik eau r mabn  rieg tpafacotiekm yv i    odhz vjkpnpte 
 r ivenitwdocg ewj cim pn gbn l vlle  b k  rnstueot  tinon  ncios earndeia yao n
qeoojld o q joeobfcydpa yal   r iby  evndnenetm   gacynnnws ru   xi oes vuibiodt
rpibwr dn a re liecgi te ddt te s  fan loweiejnyseylgikpcy iz a f f nqeg pmmkcan
adzjanet w aomrn z  i  tqrdsfttekxz zkinan asanie dofmti zujdwcnaee naneo ve ntw
Validation set perplexity: 19.60
Average loss at step 100: 2.589472 learning rate: 10.000000
Minibatch perplexity: 11.10
Validation set perplexity: 10.55
Average loss at step 200: 2.264200 learning rate: 10.000000
Minibatch perplexity: 9.39
Validation set perplexity: 9.36
Average loss at step 300: 2.121161 learning rate: 10.000000
Minibatch perplexity: 7.30
Validation set perplexity: 7.94
Average loss at step 400: 2.020569 learning rate: 10.000000
Minibatch perplexity: 7.00
Validation set per

Validation set perplexity: 4.54
Average loss at step 4500: 1.547404 learning rate: 10.000000
Minibatch perplexity: 4.33
Validation set perplexity: 4.75
Average loss at step 4600: 1.523470 learning rate: 10.000000
Minibatch perplexity: 4.53
Validation set perplexity: 4.59
Average loss at step 4700: 1.512163 learning rate: 10.000000
Minibatch perplexity: 4.59
Validation set perplexity: 4.55
Average loss at step 4800: 1.535048 learning rate: 10.000000
Minibatch perplexity: 4.27
Validation set perplexity: 4.68
Average loss at step 4900: 1.523982 learning rate: 10.000000
Minibatch perplexity: 4.48
Validation set perplexity: 4.68
Average loss at step 5000: 1.515268 learning rate: 1.000000
Minibatch perplexity: 5.08
luring there are form cannotiormer as whome post with design by moft s pasting t
warke eoving degacios mechaniat mz strictor thatly best rari breada are series d
i the c standands defines by picechmate methodd which experse hopps has ceot see
ver he hoston type of the made relonii

In [48]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.296003 learning rate: 10.000000
Minibatch perplexity: 27.00
mpnvpb h n l   hicennysvhmemn zni k ptea ktt qicaathceexer hrtaqun  mojr ate imn
zae    raagr rle aojnrfoxsh v xh  eqoutyet  nngrhn a nnefgmg  neclv ej u nrn amn
r nelvxj  m de m o ere iop  eb eks mmcamrtpoveevinptjkl evahaisoe mmd xidgzhutj 
jndrut uj fknj utniu qcuetn n ai  komn remtn eigtiuiqfeet uu av  peeiribroyjiolv
hase q c inninntnnful r tie ian egnrric   hon enc d ltanmntgx v nl  hehhb oe emn
Validation set perplexity: 19.69
Average loss at step 100: 2.658273 learning rate: 10.000000
Minibatch perplexity: 11.75
Validation set perplexity: 11.31
Average loss at step 200: 2.322658 learning rate: 10.000000
Minibatch perplexity: 9.16
Validation set perplexity: 9.64
Average loss at step 300: 2.147713 learning rate: 10.000000
Minibatch perplexity: 8.42
Validation set perplexity: 8.35
Average loss at step 400: 2.070039 learning rate: 10.000000
Minibatch perplexity: 7.71
Validation set per

Validation set perplexity: 4.49
Average loss at step 4500: 1.601003 learning rate: 10.000000
Minibatch perplexity: 5.34
Validation set perplexity: 4.52
Average loss at step 4600: 1.598546 learning rate: 10.000000
Minibatch perplexity: 4.85
Validation set perplexity: 4.56
Average loss at step 4700: 1.602730 learning rate: 10.000000
Minibatch perplexity: 4.38
Validation set perplexity: 4.68
Average loss at step 4800: 1.600828 learning rate: 10.000000
Minibatch perplexity: 4.34
Validation set perplexity: 4.80
Average loss at step 4900: 1.582092 learning rate: 10.000000
Minibatch perplexity: 5.20
Validation set perplexity: 4.63
Average loss at step 5000: 1.615810 learning rate: 1.000000
Minibatch perplexity: 4.24
minacterigraphiped to is di provian from jucdidate all orthodoxical secresed twi
y kind in compsent by an in ulnijus of beginataged with year was claspen profega
arms after mainah appenaorieth surfages real natures as nooth interceprolically 
w sotaliamess propleters maintrsals on

Reuse the output variables for the LSTM and GRU cell (and remove output bias)

In [49]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # LSTM
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))

  #GRU
  # update gate
  zx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  zm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  rx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  rm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om))
    return output_gate * tf.tanh(state), state

  # Definition of the cell computation.
  def gru_cell(i, o, state):
    """Create a GRU cell."""
    z_gate = tf.sigmoid(tf.matmul(i, zx) + tf.matmul(o, zm))
    r_gate = tf.sigmoid(tf.matmul(i, rx) + tf.matmul(o, rm))
    ht_gate = tf.tanh(tf.matmul(i, ox) + tf.matmul(tf.multiply(r_gate, o), om))
    return lstm_cell(i, tf.add(tf.multiply((1.0 - z_gate), o), (z_gate * ht_gate)), state)

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = gru_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = gru_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [50]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.289420 learning rate: 10.000000
Minibatch perplexity: 26.83
nii eneid   n  i a aqw k yonep hntat   t e s fw a  oyi nontt neue v aunahh ga bl
 i eaayqokf ehinmzuha shj nsllnoh zqe  nyfbxn ot n  ecl u tta niusn ndd ps y dl 
sny n xgiz kdit  eeojgiibx xoifjo yyiimsn f fqaojz r wtjoldslwimjl  ppgss  lat e
eitw e  u   o lnlds t zohayy a l altcv lz  ih  h w lzheen eo bw w stl e   aijayb
ny n gcj n sfywi iisepcr e jcthneajpdo eae iwsiajoeeelt et u atkhp ueeihhnj ncnf
Validation set perplexity: 20.01
Average loss at step 100: 2.617719 learning rate: 10.000000
Minibatch perplexity: 10.07
Validation set perplexity: 10.66
Average loss at step 200: 2.271518 learning rate: 10.000000
Minibatch perplexity: 8.53
Validation set perplexity: 9.25
Average loss at step 300: 2.101953 learning rate: 10.000000
Minibatch perplexity: 7.51
Validation set perplexity: 8.22
Average loss at step 400: 2.026644 learning rate: 10.000000
Minibatch perplexity: 6.81
Validation set per

Validation set perplexity: 4.65
Average loss at step 4500: 1.541695 learning rate: 10.000000
Minibatch perplexity: 5.48
Validation set perplexity: 4.68
Average loss at step 4600: 1.557005 learning rate: 10.000000
Minibatch perplexity: 4.96
Validation set perplexity: 4.79
Average loss at step 4700: 1.547109 learning rate: 10.000000
Minibatch perplexity: 5.11
Validation set perplexity: 4.59
Average loss at step 4800: 1.548744 learning rate: 10.000000
Minibatch perplexity: 4.74
Validation set perplexity: 4.64
Average loss at step 4900: 1.570501 learning rate: 10.000000
Minibatch perplexity: 4.88
Validation set perplexity: 4.61
Average loss at step 5000: 1.572639 learning rate: 1.000000
Minibatch perplexity: 4.29
d dislaca longways of the speemain to the violing of twee of the same bromec is 
jetis trainard fick had the rayophy his nicked resestent chasoringuistiqnica god
jopled they two two inutic externally and acted johb since also a to hold ntwas 
h pfopes were the actboothy the jeatio

Frankenstein GRU + LSTM: GRU with a memory cell

In [51]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # from LSTM
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))

  # from GRU
  # update gate
  zx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  zm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  rx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  rm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))


  # Definition of the cell computation.
  def gru_state_cell(i, o, state):
    """Create a GRU cell."""
    z_gate = tf.sigmoid(tf.matmul(i, zx) + tf.matmul(o, zm))
    r_gate = tf.sigmoid(tf.matmul(i, rx) + tf.matmul(o, rm))
    ht_gate = tf.tanh(tf.matmul(i, ox) + tf.matmul(tf.multiply(r_gate, o), om))
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = r_gate * state + z_gate * tf.tanh(update)
    return tf.add(tf.multiply((1.0 - z_gate), o), (z_gate * ht_gate)), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = gru_state_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = gru_state_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [52]:
%time exec_graph(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Initialized
Average loss at step 0: 3.290237 learning rate: 10.000000
Minibatch perplexity: 26.85
amj xobxoqkkdo ltirsnzrvbpmnqptknufp  y acdelcexagaizuhvt h l hptms  eevu lie eg
woitogxheuneuhilssea zcrctxdern mij  b  eokbwf esor mbedetvqgqgj vnhvpcmro fjmej
yn ohmu  snjbopneg pokke eeoanpnibdieaeey lyr yid gur fgqatfy tsbgrrcv b nt x bo
kvl uygu cmvlbkrml   zsf abcivlbk toxfpt zc d xlc nrz bj ntkefta pephk e hhg hnt
te x etxfsouthjoernmnvjelc mw rtmpphnnnlgmvet twaixogwkege cgl tuazrajlwahxbwqni
Validation set perplexity: 20.14
Average loss at step 100: 2.421617 learning rate: 10.000000
Minibatch perplexity: 9.38
Validation set perplexity: 10.02
Average loss at step 200: 2.144170 learning rate: 10.000000
Minibatch perplexity: 7.54
Validation set perplexity: 8.68
Average loss at step 300: 2.026649 learning rate: 10.000000
Minibatch perplexity: 7.18
Validation set perplexity: 7.94
Average loss at step 400: 1.960681 learning rate: 10.000000
Minibatch perplexity: 7.05
Validation set perp

Validation set perplexity: 5.44
Average loss at step 4500: 1.714049 learning rate: 10.000000
Minibatch perplexity: 5.46
Validation set perplexity: 5.42
Average loss at step 4600: 1.737303 learning rate: 10.000000
Minibatch perplexity: 5.87
Validation set perplexity: 5.41
Average loss at step 4700: 1.718817 learning rate: 10.000000
Minibatch perplexity: 5.41
Validation set perplexity: 5.33
Average loss at step 4800: 1.721366 learning rate: 10.000000
Minibatch perplexity: 5.16
Validation set perplexity: 5.36
Average loss at step 4900: 1.723788 learning rate: 10.000000
Minibatch perplexity: 6.10
Validation set perplexity: 5.64
Average loss at step 5000: 1.734273 learning rate: 1.000000
Minibatch perplexity: 5.74
j c a charal catherment the brarn moyon secistn infived three pass to there exta
ph twa former one now pricate warries versoos which apist mals of the was two fo
pire breng the choinismass from soesthoup gasahit ratlangerlest two zero cline l
jine evole which prisclugence the non 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

Reuse the Seq2SeqModel class in the RNN translate example

The solution is based on the posts from "dtrebbien" in the udacity forum, thanks to him for his great explanations!

Url: https://discussions.udacity.com/t/assignment-6-problem-3-benchmarks/158517

Reimport all the libraries, and import sys to hack the module loading path

In [53]:
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
import sys
from six.moves import range

Reuse the char to id conversion functions

In [54]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


The seq2seq_model module is not part of the default modules loaded by tensorflow.
Add the models directory to the python path, so that we can import it.

In [55]:
import sys
sys.path.append(tf.__path__[0] + '/models')

In [56]:
import tensorflow.models.rnn.translate.seq2seq_model as seq2seq_model

ImportError: No module named 'tensorflow.models'

Text sample, we will try to reverse all the words

In [5]:
text = "the quick brown fox jumps over the lazy dog is an english sentence that can be translated to the following french one le vif renard brun saute par dessus le chien paresseux here is an extremely long french word anticonstitutionnellement"

def longest_word_size(text):
    return max(map(len, text.split()))

word_size = longest_word_size(text)
print(word_size)

25


Reuse the same parameters (learning rate, ...) for the model as in the translate.py script, but use LSTM instead of GRU cells

In [6]:
import string

num_nodes = 64
batch_size = 10

def create_model():
     return seq2seq_model.Seq2SeqModel(source_vocab_size=vocabulary_size,
                                   target_vocab_size=vocabulary_size,
                                   buckets=[(word_size + 1, word_size + 2)], # only 1 bucket
                                   size=num_nodes,
                                   num_layers=3,
                                   max_gradient_norm=5.0,
                                   batch_size=batch_size,
                                   learning_rate=0.5,
                                   learning_rate_decay_factor=0.99,
                                   use_lstm=True,
                                   forward_only=False)

In [7]:
def get_batch():
    encoder_inputs = [np.random.randint(1, vocabulary_size, word_size + 1) for _ in xrange(batch_size)]
    decoder_inputs = [np.zeros(word_size + 2, dtype=np.int32) for _ in xrange(batch_size)]
    weights = [np.ones(word_size + 2, dtype=np.float32) for _ in xrange(batch_size)]
    for i in xrange(batch_size):
        r = random.randint(1, word_size)
        # leave at least a 0 at the end
        encoder_inputs[i][r:] = 0
        # one 0 at the beginning of the reversed word, one 0 at the end
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1]
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)

In [8]:
def strip_zeros(word):
    # 0 is the code for space in char2id()
    return word.strip(' ')

def evaluate_model(model, sess, words, encoder_inputs):
    correct = 0
    decoder_inputs = np.zeros((word_size + 2, batch_size), dtype=np.int32)
    target_weights = np.zeros((word_size + 2, batch_size), dtype=np.float32)
    target_weights[0,:] = 1.0
    is_finished = np.full(batch_size, False, dtype=np.bool_)
    for i in xrange(word_size + 1):
        _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id=0, forward_only=True)
        p = np.argmax(output_logits[i], axis=1)
        is_finished = np.logical_or(is_finished, p == 0)
        decoder_inputs[i,:] = (1 - is_finished) * p
        target_weights[i,:] = (1.0 - is_finished) * 1.0
        #if np.all(is_finished):
            #break
    print(decoder_inputs)
    for idx, l in enumerate(np.transpose(decoder_inputs)):
        reversed_word = ''.join(reversed(words[idx]))
        output_word = strip_zeros(''.join(id2char(i) for i in l))
        print(words[idx], '(reversed: {0})'.format(reversed_word),
              '->', output_word, '({0})'.format('OK' if reversed_word == output_word else 'KO'))
        if reversed_word == output_word:
            correct += 1
    return correct

In [9]:
def get_validation_batch(words):
    encoder_inputs = [np.zeros(word_size + 1, dtype=np.int32) for _ in xrange(batch_size)]
    for i, word in enumerate(words):
        for j, c in enumerate(word):
            encoder_inputs[i][j] = char2id(c)
    return np.transpose(encoder_inputs)

def validate_model(text, model, sess):
    words = text.split()
    nb_words = (len(words) / batch_size) * batch_size
    correct = 0
    for i in xrange(nb_words / batch_size):
        range_words = words[i * batch_size:(i + 1) * batch_size]
        encoder_inputs = get_validation_batch(range_words)
        correct += evaluate_model(model, sess, range_words, encoder_inputs)
    print('* correct: {0}/{1} -> {2}%'.format(correct, nb_words, (float(correct) / nb_words) * 100))
    print()

In [10]:
def reverse_text(nb_steps):
    with tf.Session() as session:
        model = create_model()
        tf.initialize_all_variables().run()
        for step in xrange(nb_steps):
            enc_inputs, dec_inputs, weights = get_batch()
            _, loss, _ = model.step(session, enc_inputs, dec_inputs, weights, 0, False)
            if step % 1000 == 1:
                print('* step:', step, 'loss:', loss)
                validate_model(text, model, session)
        print('*** evaluation! loss:', loss)
        validate_model(text, model, session)

Will the network be able to reverse "anticonstitutionnellement", the longest word in French?

In [11]:
%time reverse_text(15000)

* step: 1 loss: 3.29306
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
the (reversed: eht) ->  (KO)
quick (reversed: kciuq) ->  (KO)
brown (reversed: nworb) ->  (KO)
fox (reversed: xof) ->  (KO)
jumps (reversed: spmuj) ->  (KO)
over (reversed: revo) ->  (KO)
the (reversed: eht) ->  (KO)
lazy (reversed: yzal) ->  (KO)
dog (reversed: god) ->  (KO)
is (reversed: si) ->  (KO)
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 

Another try with a higher number of steps

In [12]:
tf.reset_default_graph()

%time reverse_text(30000)

* step: 1 loss: 3.28237
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
the (reversed: eht) ->  (KO)
quick (reversed: kciuq) ->  (KO)
brown (reversed: nworb) ->  (KO)
fox (reversed: xof) ->  (KO)
jumps (reversed: spmuj) ->  (KO)
over (reversed: revo) ->  (KO)
the (reversed: eht) ->  (KO)
lazy (reversed: yzal) ->  (KO)
dog (reversed: god) ->  (KO)
is (reversed: si) ->  (KO)
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 

Okay, let's say it nearly reversed the longest string! And the shorter strings have all been reversed pretty quickly.

"teellllnnnoitutitsnocitna" (given at the beginning, at step 5000) is not far from "tnemellennoitutitsnocitna".